<H1>LOAD MAIL AND PUT INTO EXCEL</H1>


In [ ]:
from load_mail import main

main()

<H1>COMBINE ALL EXCEL</H1>

In [8]:
import openpyxl
import os
import pandas as pd
from datetime import date, datetime, timedelta

############### DAYS #############
# if monday, set 3 = friday, 2 = saturday, 1 = sunday
daystime = 1

EXCEL_READ_FILEPATH = r"D:\Workstuff\my-work-python-script\download_mail_to_folder\xlsx"
EXCEL_WRITE_FILEPATH = r"D:\Workstuff\my-work-python-script\download_mail_to_folder\Result_archiev"
yesterday = date.today() - timedelta(days=daystime)

In [9]:
#Create excel
os.chdir(EXCEL_WRITE_FILEPATH)
new_wb = openpyxl.Workbook()
new_wb.save(f"Parcel Pickup {yesterday}.xlsx")
load_new_wb = openpyxl.load_workbook(os.path.join(EXCEL_WRITE_FILEPATH, f"Parcel Pickup {yesterday}.xlsx"), data_only=True)
sheet = load_new_wb['Sheet']
sheet.cell(row=1,column=1).value = "Tracking"
sheet.cell(row=1,column=2).value = "Time"
sheet.title = f"{yesterday}"
load_new_wb.save(filename=f"Parcel Pickup {yesterday}.xlsx")


In [10]:
#Combine all into one
import warnings

os.chdir(EXCEL_READ_FILEPATH)
with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    for file in os.listdir(EXCEL_READ_FILEPATH):
        data = pd.read_excel(file, dtype=object) ##turn excel into dataframe
        with pd.ExcelWriter(f"..//Result_archiev//Parcel Pickup {yesterday}.xlsx", mode="a", if_sheet_exists="overlay", engine="openpyxl") as excel_file:
            excel_file.number_format = openpyxl.styles.numbers.FORMAT_TEXT
            data.to_excel(excel_file, sheet_name=f"{yesterday}", header=False, index=False, startrow=excel_file.sheets[f"{yesterday}"].max_row)


In [11]:
#Load file and convert time to 24h format
#time convert algorithm

def convert24(str1):
    #check if am and equal 12
    if str1[-2:] == "AM" and str1[:2] == "12":
        return "00" + str1[2:-2]

    #otherwise just remove AM
    elif str1[-2:] == "AM":
        return str1[:-2]
    
    #but if it's pm and equal 12
    elif str1[-2:] == "PM" and str1[:2] == "12":
        return str1[:-2]

    else:
        return str(int(str1[:2]) + 12) + str1[2:5]

os.chdir(EXCEL_WRITE_FILEPATH)
load_time = pd.read_excel(f"Parcel Pickup {yesterday}.xlsx")
shipment_number = pd.DataFrame(load_time['Tracking'])

time_column_shipment = pd.DataFrame(load_time['Time'].str.split("([\d][\d][:][\d][\d] [A-Z][A-Z])", expand=True, regex=True))
frames = [shipment_number, time_column_shipment]
new_data = pd.concat(frames, axis=1)
with pd.ExcelWriter(f"Parcel Pickup {yesterday}.xlsx", mode="a", if_sheet_exists="replace", engine="openpyxl") as excel_file:
    new_data.to_excel(excel_file,sheet_name=f"{yesterday}", header=False, index=False, startrow=1)

#time_column = pd.DataFrame(excel_file['Time'].str.split(" ", expand = True))

<h1>LOAD ALL DATA FROM STOCKOUT</h1>

In [12]:
#stockout

stockout = pd.read_clipboard(sep='\t', dtype={'0' : 'string','16' : 'string', '11' : 'string', '15' : 'string', '18' : 'string', '21' : 'string'})
stockoutid = stockout["Stock Out (ID)"]
branchid = stockout["Branch (ID)"]
booking_id = stockout["Booking ID"].str.rstrip().str.replace("Booking-DHL ID : ","") #.str.split(" , ", expand=True)

frames = [stockoutid, branchid, booking_id]
data = pd.concat(frames, axis=1)

data = data.set_index(['Stock Out (ID)', 'Branch (ID)'])
data = data['Booking ID'].str.split(' , ', expand=True).stack().reset_index(level=2, drop=True).reset_index(name='Booking ID')
data = pd.DataFrame(data)

# group the DataFrame by columns ID2 and ID3, count the number of occurrences, and reset the index
counts_df = data.groupby(['Stock Out (ID)', 'Branch (ID)']).size().reset_index(name='Count')

# merge the counts DataFrame with the original DataFrame on columns ID2 and ID3
result_df = pd.merge(data, counts_df, on=['Stock Out (ID)', 'Branch (ID)'])

# drop duplicates of ID1 and the counted columns to keep only the first occurrence of each group
data = pd.DataFrame(result_df.drop_duplicates(subset=['Stock Out (ID)', 'Branch (ID)', 'Booking ID', 'Count'], keep='first'))



C:\Users\Comseven\AppData\Local\Temp\ipykernel_5932\3566943831.py:3: DtypeWarning: Columns (3,11) have mixed types. Specify dtype option on import or set low_memory=False.
  stockout = pd.read_clipboard(sep='\t', dtype={'0' : 'string','16' : 'string', '11' : 'string', '15' : 'string', '18' : 'string', '21' : 'string'})


In [13]:
#stockout_insure

stockout_insure = pd.read_clipboard(sep='\t')
stockoutid_insure = stockout_insure["Stock Out (ID)"]
branchid_insure = stockout_insure["Branch (ID)"] #
booking_id_insure = stockout_insure["Booking ID"].str.rstrip().str.replace("Booking-DHL ID : ","") #.str.split(" , ", expand=True)

frames_insure = [stockoutid_insure, branchid_insure, booking_id_insure]
data_insure = pd.concat(frames_insure, axis=1)

data_insure = data_insure.set_index(['Stock Out (ID)', 'Branch (ID)'])
data_insure = data_insure['Booking ID'].str.split(' , ', expand=True).stack().reset_index(level=2, drop=True).reset_index(name='Booking ID')
data_insure = pd.DataFrame(data_insure)

# group the DataFrame by columns ID2 and ID3, count the number of occurrences, and reset the index
counts_df = data_insure.groupby(['Stock Out (ID)', 'Branch (ID)']).size().reset_index(name='Count')

# merge the counts DataFrame with the original DataFrame on columns ID2 and ID3
result_df = pd.merge(data_insure, counts_df, on=['Stock Out (ID)', 'Branch (ID)'])

# drop duplicates of ID1 and the counted columns to keep only the first occurrence of each group
data_insure = pd.DataFrame(result_df.drop_duplicates(subset=['Stock Out (ID)', 'Branch (ID)', 'Booking ID', 'Count'], keep='first'))

<h1>INSERT DATAFRAME TO EXCEL THEN INDEX/MATCH</h1>

In [14]:
with pd.ExcelWriter(f"Parcel Pickup {yesterday}.xlsx", mode="a", if_sheet_exists="overlay", engine="openpyxl") as excel_file:
    data.to_excel(excel_file, sheet_name="stockout")
    data_insure.to_excel(excel_file, sheet_name="stockout_insure")

load_wb = openpyxl.load_workbook(os.path.join(EXCEL_WRITE_FILEPATH, f"Parcel Pickup {yesterday}.xlsx"), data_only=True)
load_sheet = load_wb.sheetnames
shipment_data = load_wb[load_sheet[0]]
stockout_normal = load_wb[load_sheet[1]]
stockout_insurance = load_wb[load_sheet[2]]

for row in range(2,shipment_data.max_row+1):
    #convert time
    shipment_data.cell(row=row, column=3).value = convert24(shipment_data.cell(row=row, column=3).value)
    #Stockout
    shipment_data.cell(row=row, column=4).value = f'''=IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),INDEX(stockout!B:B,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),INDEX(stockout_insure!B:B,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),""))'''
    #Branch ID
    shipment_data.cell(row=row, column=5).value = f'''=IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),INDEX(stockout!C:C,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),INDEX(stockout_insure!C:C,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),""))'''
    #type
    shipment_data.cell(row=row, column=6).value = f'''=IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),"Stockout",IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),"Stockout_Insure",""))'''
    #quantity
    shipment_data.cell(row=row, column=7).value = f'''=IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),INDEX(stockout!E:E,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout!D:D,0)),IF(ISNUMBER(MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),INDEX(stockout_insure!E:E,MATCH({openpyxl.utils.quote_sheetname(shipment_data.title)}!A{row},stockout_insure!D:D,0)),""))'''

load_wb.save(f"Parcel Pickup {yesterday}.xlsx")

In [ ]:
##LEGACY crap

import win32com.client as win32

def openWorkbook(xlapp, xlfile):
    try:
        xlwb = xlapp.Workbooks(xlfile)
    except Exception as e:
        try:
            xlwb = xlapp.Workbooks.Open(xlfile)
        except Exception as e:
            print(e)
            xlwb = None
    return(xlwb)

try:
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = openWorkbook(excel, os.path.join(EXCEL_READ_FILEPATH, f"Parcel Pickup {yesterday}.xlsx"))
    ws = wb.Worksheets(yesterday)
    excel.Visible = True

except Exception as e:
    print(e)

finally:
    ws = None
    wb = None
    excel = None

In [ ]:
data_insure